<a href="https://colab.research.google.com/github/jpcompartir/dl_notebooks/blob/main/hf_sent_alg_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.19.2

skeleton notebook for running sent alg.

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline 
from google.colab import files 
import numpy as np 
import torch

import io
from io import BytesIO 

In [ ]:
data = files.upload()

In [ ]:
def share_sent(df, auth_token = "censored", model_checkpoint = "jpcompartir/share_sent_es", batch_size = 64, text_variable = "text"):
  
  device = "cuda:0" if torch.cuda.is_available() else "cpu"

  texts = df[text_variable].tolist()
  original_id = df["original_id"].tolist()

  auth_token = auth_token
  model_checkpoint = model_checkpoint
  
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_auth_token = auth_token, model_max_length = 128, truncation = True)

  model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, use_auth_token = auth_token)
  model.to(device)

  pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores = True, device = 0, truncation = True, batch_size = batch_size)

  results = pipe(texts)
  results_df = pd.DataFrame.from_dict(results).apply(lambda x: pd.Series([dict["score"] for dict in x])).rename(columns = {0: "positive", 1:"negative", 2: "neutral"})
  results_df["py_document"] = np.arange(1, len(results) + 1, 1)
  results_df["text"] = texts
  results_df["original_id"] = original_id

  
  return results_df[["original_id", "py_document", "text", "positive", "negative", "neutral"]]

In [ ]:
results_one = share_sent(df)

In [ ]:
results_one.head()

In [ ]:
results_one_csv = results_one.to_csv("results_one.csv", index = False)

In [ ]:
files.download("results_one.csv")